# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data = pd.read_csv("../WeatherPy/cities.csv")
cities_df = pd.DataFrame(cities_data)
cities_df.head(50)

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Kapaa,90,US,1612801077,82,22.0752,-159.3190,62.60,9.22
1,1,Port-Cartier,100,CA,1612801077,74,50.0334,-66.8654,25.00,9.22
2,2,Kungurtug,24,RU,1612801077,86,50.5994,97.5228,10.33,3.60
3,3,Tuktoyaktuk,1,CA,1612801077,74,69.4541,-133.0374,-40.00,6.38
4,4,Aklavik,100,CA,1612801077,67,68.2191,-135.0107,-36.40,3.44
5,5,Mamurras,90,AL,1612801077,87,41.5775,19.6922,53.60,3.44
6,6,Avarua,75,CK,1612801077,83,-21.2078,-159.7750,80.60,4.61
7,7,Saint Anthony,1,US,1612801078,64,45.0205,-93.2180,-5.80,3.51
8,8,Ekuvukeni,61,ZA,1612801079,79,-28.4675,30.1551,72.99,11.56
9,9,Ahipara,100,NZ,1612801079,87,-35.1667,173.1667,64.40,6.33


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configuring gmaps
gmaps.configure(api_key=g_key)

In [4]:
# creating the locations and weight from the cities_df
locations = cities_df[["Lat", "Lng"]].astype(float)
my_weights = cities_df["Humidity"].astype(float)

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=my_weights, dissipating=False, max_intensity=100,
                                point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Ideal conditions: Temp between 70-80, wind speed < 10, cloudiness = 0 
# store into a variable called hotel_df
hotel_df = pd.DataFrame()
hotel_df = cities_df.loc[(cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 90) 
                                    & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0)] 

hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
40,40,Hermanus,0,ZA,1612801082,73,-34.4187,19.2345,75.99,5.01
48,48,Cidreira,0,BR,1612800609,61,-30.1811,-50.2056,76.24,9.51
59,59,Champerico,0,GT,1612801085,66,14.3000,-91.9167,86.00,3.04
83,83,Castro,0,BR,1612801087,51,-24.7911,-50.0119,74.95,4.65
118,118,Grand Gaube,0,MU,1612801092,76,-20.0064,57.6608,81.00,8.01
124,124,Safaga,0,EG,1612801093,33,26.7292,33.9365,75.20,3.44
207,207,Coyhaique,0,CL,1612801103,31,-45.5752,-72.0662,75.20,3.44
214,214,Bredasdorp,0,ZA,1612801104,47,-34.5322,20.0403,80.60,9.22
235,235,Binabalian Ricor,0,PH,1612801107,74,16.4202,119.9279,79.75,8.63
245,245,Río Bueno,0,CL,1612801108,41,-40.3167,-72.9667,80.60,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# adding new column to hotel_df
hotel_df["Hotel Name"] = ""

/Users/EricShaffer/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
# create the base url for the api search 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "rankby": "distance",
    "type": "airport",
    "keyword": "international airport",
    "key": g_key
}

In [1]:
# make successive api calls for each city and find the nearest hotel within 5000 meters
for index, row in hotel_df.iterrows():

    #assign the location of the city (lat and lng)
    city_coord = f"{row['Lat'], row['Lng']}"
    
    # add this to the api
    params["location"] = city_coord
    
    # make the api call
    response = requests.get(base_url, params=params).json()
    
    time.sleep(1)
        
    results = response['results']
    

    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        print(f'The nearest hotel to {row["City"]} is {results[0]["name"]}')
    except:
        hotel_df.loc[index, "Hotel Name"] = ""
        pass
    

NameError: name 'hotel_df' is not defined

In [22]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
40,40,Hermanus,0,ZA,1612801082,73,-34.4187,19.2345,75.99,5.01,
48,48,Cidreira,0,BR,1612800609,61,-30.1811,-50.2056,76.24,9.51,
59,59,Champerico,0,GT,1612801085,66,14.3000,-91.9167,86.00,3.04,
83,83,Castro,0,BR,1612801087,51,-24.7911,-50.0119,74.95,4.65,
118,118,Grand Gaube,0,MU,1612801092,76,-20.0064,57.6608,81.00,8.01,
124,124,Safaga,0,EG,1612801093,33,26.7292,33.9365,75.20,3.44,
207,207,Coyhaique,0,CL,1612801103,31,-45.5752,-72.0662,75.20,3.44,
214,214,Bredasdorp,0,ZA,1612801104,47,-34.5322,20.0403,80.60,9.22,
235,235,Binabalian Ricor,0,PH,1612801107,74,16.4202,119.9279,79.75,8.63,
245,245,Río Bueno,0,CL,1612801108,41,-40.3167,-72.9667,80.60,5.75,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
